In [0]:
#if not running on Google Colabs, this code snippet is unnecessary

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-driv/e-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmprtelnync/pubring.gpg' created
gpg: /tmp/tmprtelnync/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
#if not running on Google Colabs, this code snippet is unnecessary

!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#address to the root directory
address = '/content/drive/App/Language_Identification'

In [0]:
import os
os.chdir(address)

In [0]:
f = open('DSLCC-v2.0/train-dev/train.txt','r')            # Dataset consists of 13 languages with 18,000 training sentences and 2,000 test
text_data = f.read()                                      # sentences for each language. To emulate a realistic language identification
lines = text_data.splitlines()                            # setup additional texts from various languages are added to the data set.
f.close()                                                 # They are labelled with the language code (xx).

In [0]:
fraction = 10                      # due to memory limitations, only a fraction of the dataset is used for training
timesteps = 30                     # As we are training an LSTM model, we consider the first 50 characters in the sentence to predict the language of text
hidden_units = 100
epochs = 100
batch_size = 50

In [0]:
import re
labels = []
sentences = []
for line_index,line in enumerate(lines) :
  if line_index == 206918:
    continue
  if line_index%fraction == 0 :                       # only a fraction of the dataset is considered for training
    temp = line.split('\t')                           # each line in the dataset is of the form ----> "sentence<tab space>label"
    sentences.append(temp[0])
    labels.append(temp[1])

In [0]:
import numpy as np
num = np.zeros(100000)                                # 100000 is chosen arbitrarily just as an upperbound to compute the
for sentence in sentences:                            # dictionaries for character encodings and decodings
  for ch in sentence:
    if num[ord(ch)] == 0 :
      num[ord(ch)] = 1
char_decoding = []
for ch_num,x in enumerate(num) :
  if x == 1 :
    char_decoding.append(ch_num)
char_encoding = {}
for i,ch_num in enumerate(char_decoding) :
  char_encoding[ch_num] = i

In [0]:
len(char_encoding)

278

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
sent_num = []
for sentence in sentences :
  temp = []
  for character in sentence[:timesteps] :                               # only the first 'timesteps' number of characters are considered for training 
    temp.append(char_encoding[ord(character)])                          # characters are stored in their unicode format
    
  temp = np.asarray(temp)
  input_onehot_encoder = OneHotEncoder(n_values=len(char_encoding),sparse = False)
  temp = temp.reshape(len(temp), 1)
  input_onehot_encoded = input_onehot_encoder.fit_transform(temp)       # we convert the character unicode list to one hot vectors
                                                                        # of shape [fractioned_dataset_size, timesteps, n_values]
  sent_num.append(input_onehot_encoded)
inputs = np.asarray(sent_num)
inputs.shape

(25201, 30, 278)

In [0]:
from collections import Counter                                         # it can be seen that the dataset is not biased with
print(Counter(labels))                                                  # equal distribution for all the languages                                                                        

Counter({'sk': 1801, 'bg': 1800, 'bs': 1800, 'cz': 1800, 'es-AR': 1800, 'es-ES': 1800, 'hr': 1800, 'id': 1800, 'mk': 1800, 'my': 1800, 'pt-BR': 1800, 'pt-PT': 1800, 'sr': 1800, 'xx': 1800})


In [0]:
print(inputs.shape)

(25201, 30, 278)


In [0]:
label_encoder = LabelEncoder()                                                       # 'labels' consisted of 14 different languages
label_integer_encoded = label_encoder.fit_transform(labels)                          # 'label_integer_encoded' encodes labels into numbers, 0-13

label_onehot_encoder = OneHotEncoder(sparse=False)
label_integer_encoded = label_integer_encoded.reshape(len(label_integer_encoded), 1)
label_hot = label_onehot_encoder.fit_transform(label_integer_encoded)                # labels_hot has dimensions [fractioned_dataset_size, 14]

#inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])

In [0]:
print(label_hot.shape)

(25201, 14)


In [0]:
from sklearn.model_selection import train_test_split
random_seed = 2
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(inputs, label_hot, test_size = 0.1, random_state=random_seed)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(timesteps, len(char_encoding)), return_sequences=True, dropout = 0.25))  # Use of recurrent_dropout has deleterious on training accuracy
model.add(LSTM(hidden_units, dropout = 0.25))                                                                      # However, standard dropout still has a good regularising effect and prevents overfitting
model.add(Dense(14, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 100)           151600    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 14)                1414      
Total params: 233,414
Trainable params: 233,414
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# train
model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=2, validation_data = (X_val,Y_val))

Train on 22680 samples, validate on 2521 samples
Epoch 1/100
 - 72s - loss: 1.7150 - val_loss: 1.3140
Epoch 2/100
 - 72s - loss: 1.3416 - val_loss: 1.1597
Epoch 3/100
 - 72s - loss: 1.2612 - val_loss: 1.1126
Epoch 4/100
 - 72s - loss: 1.1985 - val_loss: 1.0714
Epoch 5/100
 - 72s - loss: 1.1662 - val_loss: 1.0508
Epoch 6/100
 - 73s - loss: 1.1329 - val_loss: 1.0144
Epoch 7/100
 - 72s - loss: 1.1111 - val_loss: 1.0191
Epoch 8/100
 - 72s - loss: 1.0967 - val_loss: 1.0372
Epoch 9/100
 - 72s - loss: 1.0694 - val_loss: 0.9887
Epoch 10/100
 - 72s - loss: 1.0507 - val_loss: 1.0021
Epoch 11/100
 - 72s - loss: 1.0343 - val_loss: 0.9648
Epoch 12/100
 - 71s - loss: 1.0193 - val_loss: 0.9519
Epoch 13/100
 - 72s - loss: 0.9967 - val_loss: 0.9545
Epoch 14/100
 - 72s - loss: 0.9755 - val_loss: 0.9303
Epoch 15/100
 - 71s - loss: 0.9559 - val_loss: 0.9013
Epoch 16/100
 - 72s - loss: 0.9416 - val_loss: 0.9027
Epoch 17/100
 - 71s - loss: 0.9230 - val_loss: 0.8686
Epoch 18/100
 - 71s - loss: 0.9078 - val_l

Epoch 43/100
 - 72s - loss: 0.6888 - val_loss: 0.7732
Epoch 44/100
 - 72s - loss: 0.6846 - val_loss: 0.7721
Epoch 45/100
 - 72s - loss: 0.6819 - val_loss: 0.7519
Epoch 46/100
 - 71s - loss: 0.6758 - val_loss: 0.7440
Epoch 47/100
 - 71s - loss: 0.6708 - val_loss: 0.7556
Epoch 48/100
 - 71s - loss: 0.6609 - val_loss: 0.7494
Epoch 49/100
 - 70s - loss: 0.6525 - val_loss: 0.7661
Epoch 50/100
 - 70s - loss: 0.6534 - val_loss: 0.7358
Epoch 51/100
 - 70s - loss: 0.6481 - val_loss: 0.7683
Epoch 52/100
 - 69s - loss: 0.6413 - val_loss: 0.7602
Epoch 53/100
 - 70s - loss: 0.6371 - val_loss: 0.7503
Epoch 54/100
 - 70s - loss: 0.6327 - val_loss: 0.7746
Epoch 55/100
 - 70s - loss: 0.6233 - val_loss: 0.7503
Epoch 56/100
 - 69s - loss: 0.6231 - val_loss: 0.7628
Epoch 57/100
 - 70s - loss: 0.6084 - val_loss: 0.7669
Epoch 58/100
 - 70s - loss: 0.6071 - val_loss: 0.7550
Epoch 59/100
 - 70s - loss: 0.6038 - val_loss: 0.7722
Epoch 60/100
 - 71s - loss: 0.6004 - val_loss: 0.7448
Epoch 61/100
 - 70s - loss: 

 - 71s - loss: 0.4925 - val_loss: 0.7979
Epoch 86/100
 - 70s - loss: 0.4902 - val_loss: 0.8047
Epoch 87/100
 - 70s - loss: 0.4864 - val_loss: 0.8051
Epoch 88/100
 - 71s - loss: 0.4818 - val_loss: 0.8025
Epoch 89/100
 - 72s - loss: 0.4760 - val_loss: 0.8316
Epoch 90/100
 - 72s - loss: 0.4753 - val_loss: 0.8058
Epoch 91/100
 - 72s - loss: 0.4706 - val_loss: 0.8006
Epoch 92/100
 - 72s - loss: 0.4645 - val_loss: 0.8060
Epoch 93/100
 - 71s - loss: 0.4625 - val_loss: 0.8077
Epoch 94/100
 - 71s - loss: 0.4637 - val_loss: 0.8274
Epoch 95/100
 - 71s - loss: 0.4516 - val_loss: 0.8393
Epoch 96/100
 - 71s - loss: 0.4560 - val_loss: 0.8569
Epoch 97/100
 - 71s - loss: 0.4461 - val_loss: 0.8758
Epoch 98/100
 - 71s - loss: 0.4472 - val_loss: 0.8382
Epoch 99/100
 - 71s - loss: 0.4420 - val_loss: 0.8413
Epoch 100/100
 - 71s - loss: 0.4436 - val_loss: 0.8363


In [0]:
# A random Czech (cz) sentence taken from internet.
text = "Dobré ráno. Dobrý večer. Děkuji (pěkně). Děkuji (pěkně). Co znamená “lingua” v angličtině? Potřebuji procvičovat svou češtinu. Moje čeština je špatná."

temp = []
for character in text[:timesteps] :                                
  temp.append(char_encoding[ord(character)])                          

temp = np.asarray(temp)
input_onehot_encoder = OneHotEncoder(n_values=len(char_encoding),sparse = False)
temp = temp.reshape(len(temp), 1)
input_onehot_encoded = input_onehot_encoder.fit_transform(temp)       
X_test = np.asarray(input_onehot_encoded)
X_test = X_test.reshape(1,X_test.shape[0],X_test.shape[1])
print(X_test.shape)
Y_pred = model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred) 
print(Y_pred_classes)
inverted = label_encoder.inverse_transform(Y_pred_classes)
print(inverted)

(1, 30, 278)
2
cz


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
# evaluate
Y_pred = model.predict(X_val)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(Y_val,axis = 1)
count = 0
for i in range(len(Y_pred)):
  if(Y_pred_classes[i] == Y_true[i]) :
    count+=1
accuracy = 100*count/len(Y_pred)
print("Accuracy : " + str(accuracy)+"%")

Accuracy : 64.49821499404997%


In [0]:
# Prediction accuracy is always below 70%. This is naturally true as we consider only 30 timesteps of each sentence, over a small dataset. 
#   -> This prevents the network from learning long range dependencies at character level. Performance can be improved with increasing the 
#      size of dataset and network size (number of hidden units and layers).
#   -> If we try to increase the number of timesteps, due to hardware limitations we are forced to operate over a fraction of the dataset. 
#   -> This model was designed with keeping hardware limitations in mind.
# Random guessing accuracy is slightly less than 8%, which is a clear proof that our method has learnt to extract character level sequence
# features for language identification.

In [0]:
!ls "/content/drive/App/Language_Identification"
#os.chdir('Models')
model.save("Slide_latest.h5")

drive  DSLCC-v2.0  Slide_100_30_100_100_6.h5  SLIDE.ipynb


In [0]:
#restore model
from keras.models import load_model
model = load_model('Slide_latest.h5')

Using TensorFlow backend.


In [0]:
# # A random Slovak (sk) sentence taken from internet. 
text = "Mám ťa rada, lebo si dobrá. Adam ide zajtra do práce sám. Adam ide zajtra do práce sám."


temp = []
for character in text[:timesteps] :                                
  temp.append(char_encoding[ord(character)])                          

temp = np.asarray(temp)
input_onehot_encoder = OneHotEncoder(n_values=len(char_encoding),sparse = False)
temp = temp.reshape(len(temp), 1)
input_onehot_encoded = input_onehot_encoder.fit_transform(temp)       
X_test = np.asarray(input_onehot_encoded)
X_test = X_test.reshape(1,X_test.shape[0],X_test.shape[1])
print(X_test.shape)
Y_pred = model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred) 
print(Y_pred_classes)
inverted = label_encoder.inverse_transform(Y_pred_classes)
print(inverted)

(1, 30, 278)
11
sk


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
# As English is not part of the list of 13 languages, the output assigned is xx
text = "We organise what we write into sentences and paragraphs. A paragraph begins on a new line within the text and there is often a blank line between paragraphs. A paragraph usually contains more than one sentence and it is usually about one topic."

temp = []
for character in text[:timesteps] :                                
  temp.append(char_encoding[ord(character)])                          

temp = np.asarray(temp)
input_onehot_encoder = OneHotEncoder(n_values=len(char_encoding),sparse = False)
temp = temp.reshape(len(temp), 1)
input_onehot_encoded = input_onehot_encoder.fit_transform(temp)       
X_test = np.asarray(input_onehot_encoded)
X_test = X_test.reshape(1,X_test.shape[0],X_test.shape[1])
print(X_test.shape)
Y_pred = model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred) 
print(Y_pred_classes)
inverted = label_encoder.inverse_transform(Y_pred_classes)
print(inverted)

(1, 30, 278)
13
xx


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
